<a href="https://colab.research.google.com/github/AngelinaMaverino/ChatBot/blob/copy-googleColab/CHATBOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot


## Importar librerias

In [1]:
!pip install sentence-transformers==3.3.0
!pip install --upgrade pinecone
!pip install accelerate==1.1.1
!pip install transformers==4.46.2
!pip install langchain-community==0.3.5
!pip install bitsandbytes==0.44.1
!pip install diffusers==0.27.2
!pip install beautifulsoup4==4.12.3

  Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl (122.4 MB)
  Using cached diffusers-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached diffusers-0.27.2-py3-none-any.whl (2.0 MB)
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.34.0
    Uninstalling diffusers-0.34.0:
      Successfully uninstalled diffusers-0.34.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.4
    Uninstalling beautifulsoup4-4.13.4:
      Successfully uninstalled beautifulsoup4-4.13.4


In [2]:
from transformers import pipeline

import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

import re

from bs4 import BeautifulSoup

import requests
from bs4 import BeautifulSoup
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
import pinecone

from pinecone import Pinecone, ServerlessSpec


## Normalizar texto

In [3]:
def normalize_text(text):
    text = text.lower()
    # Elimina caracteres especiales y multiples espacios en blanco
    text = re.sub(r'[^a-záéíóúñ0-9\s\(\):,\.]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


## Scraping de la web de ort

### Procesamiento de los enlaces dentro de la página

In [4]:
def extraer_enlaces(soup):
    enlaces = []

    # Obtenemos solo los links que se encuentran en la pagina "central", no en el footer ni header
    central_panel = soup.find('div', id='centralpanel')

    if central_panel:
        for tag in central_panel.find_all('a', href=True):
            enlaces.append(tag['href'])

    return enlaces


También se procesan los diferentes enlaces de la página para obtener más información. Esto no incluye los enlaces del pie de página (footer) ni del encabezado (header), aunque podrían ser incorporados en el futuro.

In [5]:
def procesar_enlaces(base_url, enlaces):
    documents = []

    for enlace in enlaces:
        if not enlace.startswith('http'):
            enlace = base_url + enlace

        try:
            response = requests.get(enlace)
            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')
                docs_from_link = procesar_documento(soup)
                documents.extend(docs_from_link)  # Añadir los documentos procesados a la lista
        except requests.exceptions.RequestException as e:
            print(f"Error al procesar el enlace {enlace}: {e}")

    return documents


### Procesamiento de la pagina

Desarrollamos una función que vincula los títulos con los textos correspondientes, ya que la estructura de la página, caracterizada por títulos y textos dispersos, dificultaba la comprensión del contexto. Con esta función, primero añadimos el título del div al que pertenece el texto y luego el contenido textual. De esta manera, logramos organizar mejor los textos, proporcionando un contexto más claro y una estructura más coherente.

In [6]:
def procesar_documento(soup):
    documents = []

    for section in soup.find_all('section'):
        current_header = ""
        current_chunk = ""
        current_section_title = ""

        for tag in section.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
            if tag.name.startswith('h'):
                if not current_header:
                    current_header = tag.get_text().strip()
                    current_section_title = tag.get_text().strip()
                else:
                    if current_chunk:
                        documents.append(f"{current_header}: {current_chunk.strip()}")
                        current_chunk = ""
                    current_header = current_section_title + ': ' + tag.get_text().strip()
            else:
                current_chunk += " " + tag.get_text()

        if current_chunk:
            documents.append(f"{current_header}: {current_chunk.strip()}")
            current_chunk = ""

    for tag in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p']):
        if tag.find_parent('section') is None:
            documents.append(tag.get_text().strip())

    return documents

### Scraping

In [7]:
def obtener_documentos(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        documents = procesar_documento(soup)

        enlaces = extraer_enlaces(soup)

        documents_from_links = procesar_enlaces(url, enlaces)

        documents.extend(documents_from_links)

        return documents
    else:
        print(f"Error al acceder a la URL: {response.status_code}")
        return []

In [8]:
url = "https://fi.ort.edu.uy/ingenieria-en-sistemas"

documents = obtener_documentos(url)

docs = [Document(page_content=normalize_text(doc)) for doc in documents]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

text_chunks = text_splitter.split_documents(docs)

## Creacion de embeddings

Para esto primero se probaron dos modelos diferentes para decidir cual era el mejor modelo de embeddings

### All-MiniLM-L6-v2

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipython-input-3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# Extraer texto de los chunks
texts = [chunk.page_content for chunk in text_chunks]  # Lista de textos a embebedar

# Crear embeddings usando el modelo con dimensión 384
vectors = embeddings.embed_documents(texts)  # Generar los embeddings para cada chunk

## Base de datos vectorial

### Creacion de la base de datos vectorial

In [11]:
# Configuracion del API key y creacion de la instancia de Pinecone

pinecone_token = getpass.getpass("Ingresá tu Pinecone API Key: ")

# Crear la instancia de Pinecone
pc = Pinecone(api_key=pinecone_token)

index_name = "ort-5"

# Verificar si el indice ya existe
existing_indexes = pc.list_indexes().names()
if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )


# Conectar al indice
index = pc.Index(index_name)

## Almacenamiento en base de datos

In [12]:
batch_size = 1000

# Dividir los vectores en lotes y hacer upsert de cada uno
for i in range(0, len(vectors), batch_size):
    batch_vectors = vectors[i:i + batch_size]
    batch_texts = texts[i:i + batch_size]

    upserts = [(str(i + j), vector, {"content": text})
            for j, (vector, text) in enumerate(zip(batch_vectors, batch_texts))]

    # Subir el lote a Pinecone
    index.upsert(upserts)

print(f"Vector store creado con éxito en Pinecone con {len(text_chunks)} embeddings.")

Vector store creado con éxito en Pinecone con 1463 embeddings.


## Modelo de generacion de textos

In [13]:
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.1"

In [14]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",         # Lo pone en la GPU T4 automáticamente
    torch_dtype=torch.float16  # Más liviano que float32 y más compatible que 4-bit
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
# Crear el generador de texto
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [17]:
def get_relevant_context(question, top_k=25):
    # Generar embedding de la pregunta
    question_embedding = embeddings.embed_documents([question])[0]

    # Utilizamos Pinecone para realizar busqueda por similitud, que fue seteada cuando creamos la bd, utilizando la metrica dotproduct, que fue la que mejores respuestas nos dio
    results = index.query(vector=question_embedding, top_k=top_k, include_metadata=True)

    context = [match['metadata']['content'] for match in results['matches']]

    return context

In [18]:
def ask_bot(question):

    context = get_relevant_context(question)

    if not context:
        return "No se encontraron documentos relevantes para la pregunta."

    prompt = (
        f"Usa solo la siguiente información para responder de forma precisa:\n\n"
        f"Contexto:\n{context}\n\n"
        f"Pregunta: {question}\n\n"
        f"Responde de manera precisa y breve:"
    )

    # Generar la respuesta usando el contexto
    response = generator(
        prompt, max_new_tokens=500, truncation=True
    )[0]['generated_text']

    generated_answer = response.split("Responde de manera precisa y breve:")[-1].strip()

    return generated_answer

## Preguntas de prueba

In [19]:
pregunta = normalize_text("¿Cuál es la duración de años?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Respuesta: 5 años


In [20]:
pregunta =  normalize_text("¿Cuáles son los horarios de clase matutino?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Respuesta: Los horarios de clase matutino son de lunes a viernes de 8:00 a 13:00.


In [21]:
pregunta =  normalize_text("¿horarios de clase para semestre nocturno?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Respuesta: Los horarios de clase para el semestre nocturno son de lunes a jueves de 17:30 a 23:30.


In [22]:
pregunta =  normalize_text("¿horarios de clase vespertino?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Respuesta: Lunes a jueves de 12:00 a 18:00 (únicamente para semestre 1)


In [23]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno matutino del semestre 1 al 4?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Respuesta: Modalidad de cursado: presencial


In [24]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno vespertino del semestre 5 en adelante?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 186.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 102.12 MiB is free. Process 18891 has 14.64 GiB memory in use. Of the allocated memory 13.90 GiB is allocated by PyTorch, and 621.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
pregunta =  normalize_text("¿Cual es la modalidad de cursado para el turno nocturno?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que son las Áreas de profundización?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("Dame info sobre el area de profundizacion de Inteligencia Artificial y Analítica de Datos")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)


In [ ]:
pregunta = normalize_text("¿cuando se informan los valores de las cuotas?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cual es el Perfil de los graduados de ingeniería en sistemas?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Quiénes pueden cursar el CerIA?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)


In [ ]:
pregunta =  normalize_text("¿El CerIA es un título de postgrado?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿A quienes esta dirigido el Servicio de orientación laboral?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que se hace en el Servicio de orientación laboral?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Cuando la Facultad de Ingeniería recibió por parte de la Comisión ad hoc de la Acreditación ARCU-SUR?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que cargos ocupan los graduados?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)

In [ ]:
pregunta =  normalize_text("¿Que es el Taller de Nivelación de Conocimientos de Matemática?")
respuesta = ask_bot(pregunta)

print("Respuesta:", respuesta)